In [1]:
import pandas as pd
import numpy as np
import janitor as jn

In [2]:
print 'i'

In [15]:
url = "https://raw.githubusercontent.com/samukweku/data-wrangling-blog/master/notebooks/Data_files/flights.csv"
flights = pd.read_csv(url, sep = '\t', names=['orig','dest','orig_time', 'dest_time'], parse_dates  = ['orig_time', 'dest_time'])
flights = flights.factorize_columns(['orig','dest']).iloc[:, 2:]
flights.columns = [ent.split("_")[0] if ent.endswith('enc') else ent for ent in flights]
flights.columns = ['takeoff','landing','orig','dest']
flights = flights.assign(start=flights.landing+pd.Timedelta(minutes=45), end=flights.landing+pd.Timedelta(hours=3))
flights.head()

,takeoff,landing,orig,dest,start,end
0,2021-11-27 07:15:00,2021-11-27 08:55:00,0,0,2021-11-27 09:40:00,2021-11-27 11:55:00
1,2021-11-27 20:05:00,2021-11-28 00:50:00,1,1,2021-11-28 01:35:00,2021-11-28 03:50:00
2,2021-11-27 21:00:00,2021-11-27 21:35:00,2,2,2021-11-27 22:20:00,2021-11-28 00:35:00
3,2021-11-27 21:15:00,2021-11-27 22:25:00,0,2,2021-11-27 23:10:00,2021-11-28 01:25:00
4,2021-11-26 11:40:00,2021-11-26 14:45:00,3,3,2021-11-26 15:30:00,2021-11-26 17:45:00


In [23]:
%timeit flights.loc[:,['orig','takeoff']].sort_values(['orig','takeoff'])

5.74 ms ± 91.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [19]:
%%timeit 
flights.takeoff.sort_values(), flights.orig.sort_values()

10.1 ms ± 53.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [3]:
df = {
    "A": [0, 0],
    "B": [-1.0, 0.0],
    "E": pd.to_datetime(["1970-01-01", "1970-01-01"]),
}


right = {
    "Integers": [0, 1, 1],
    "Numeric": [0.0, 1.0, 0.0],
    "Dates": pd.to_datetime(
        [
            "1970-01-01 00:00:00.000000000",
            "1969-12-31 23:59:59.999999999",
            "1969-12-31 23:59:59.999999999",
        ]
    ),
    "Dates_Right": pd.to_datetime(
        [
            "1970-01-01",
            "1970-01-01",
            "1970-01-01",
        ]
    ),
}


right = pd.DataFrame(right)

df = pd.DataFrame(df)

In [4]:
df

,A,B,E
0,0,-1.0,1970-01-01
1,0,0.0,1970-01-01


In [5]:
right

,Integers,Numeric,Dates,Dates_Right
0,0,0.0,1970-01-01 00:00:00.000000000,1970-01-01
1,1,1.0,1969-12-31 23:59:59.999999999,1970-01-01
2,1,0.0,1969-12-31 23:59:59.999999999,1970-01-01


In [6]:
filters = ["A", "B", "E", "Integers", "Numeric", "Dates"]
df = df.assign(A=df["A"].astype("Int64"))
right = right.assign(Integers=right["Integers"].astype(pd.Int64Dtype()))

In [7]:
expected = df[["A", "B", "E"]].merge(
            right[["Integers", "Numeric", "Dates"]], how="cross"
        )

expected

,A,B,E,Integers,Numeric,Dates
0,0,-1.0,1970-01-01,0,0.0,1970-01-01 00:00:00.000000000
1,0,-1.0,1970-01-01,1,1.0,1969-12-31 23:59:59.999999999
2,0,-1.0,1970-01-01,1,0.0,1969-12-31 23:59:59.999999999
3,0,0.0,1970-01-01,0,0.0,1970-01-01 00:00:00.000000000
4,0,0.0,1970-01-01,1,1.0,1969-12-31 23:59:59.999999999
5,0,0.0,1970-01-01,1,0.0,1969-12-31 23:59:59.999999999


In [8]:
expected.loc[
            # expected.A.ne(expected.Integers)
             expected.B.lt(expected.Numeric)

            
            & expected.E.ge(expected.Dates),
        ]

,A,B,E,Integers,Numeric,Dates
0,0,-1.0,1970-01-01,0,0.0,1970-01-01 00:00:00.000000000
1,0,-1.0,1970-01-01,1,1.0,1969-12-31 23:59:59.999999999
2,0,-1.0,1970-01-01,1,0.0,1969-12-31 23:59:59.999999999
4,0,0.0,1970-01-01,1,1.0,1969-12-31 23:59:59.999999999


In [9]:
pd.__version__

'2.2.1'

In [10]:
(df[["A", "B", "E"]]
            .conditional_join(
                right[["Integers", "Numeric", "Dates"]],
                ("E", "Dates", ">="),
                  ("A", "Integers", "!="),
                ("B", "Numeric", "<"),
                how="inner",
                use_numba=True,
            ))

lassss


,A,B,E,Integers,Numeric,Dates
0,0,-1.0,1970-01-01,1,1.0,1969-12-31 23:59:59.999999999
1,0,-1.0,1970-01-01,1,0.0,1969-12-31 23:59:59.999999999
2,0,0.0,1970-01-01,1,1.0,1969-12-31 23:59:59.999999999


In [11]:
A =(
    df[["E"]].conditional_join(
        right[["Dates", "Dates_Right"]],
        ("E", "Dates", ">="),
        ("E", "Dates_Right", "<="),
        how="inner",
        use_numba=True,
    )
)

A

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


,E,Dates,Dates_Right
0,1970-01-01,1970-01-01 00:00:00.000000000,1970-01-01
1,1970-01-01,1969-12-31 23:59:59.999999999,1970-01-01
2,1970-01-01,1969-12-31 23:59:59.999999999,1970-01-01
3,1970-01-01,1970-01-01 00:00:00.000000000,1970-01-01
4,1970-01-01,1969-12-31 23:59:59.999999999,1970-01-01
5,1970-01-01,1969-12-31 23:59:59.999999999,1970-01-01


In [12]:
B = (
    df[["E"]].conditional_join(
        right[["Dates", "Dates_Right"]],
        ("E", "Dates", ">="),
        ("E", "Dates_Right", "<="),
        how="inner",
        use_numba=False,
    )
)

B

,E,Dates,Dates_Right
0,1970-01-01,1969-12-31 23:59:59.999999999,1970-01-01
1,1970-01-01,1969-12-31 23:59:59.999999999,1970-01-01
2,1970-01-01,1970-01-01 00:00:00.000000000,1970-01-01
3,1970-01-01,1969-12-31 23:59:59.999999999,1970-01-01
4,1970-01-01,1969-12-31 23:59:59.999999999,1970-01-01
5,1970-01-01,1970-01-01 00:00:00.000000000,1970-01-01


In [13]:
A.sort_values(["E", "Dates", "Dates_Right"], ignore_index=True)

,E,Dates,Dates_Right
0,1970-01-01,1969-12-31 23:59:59.999999999,1970-01-01
1,1970-01-01,1969-12-31 23:59:59.999999999,1970-01-01
2,1970-01-01,1969-12-31 23:59:59.999999999,1970-01-01
3,1970-01-01,1969-12-31 23:59:59.999999999,1970-01-01
4,1970-01-01,1970-01-01 00:00:00.000000000,1970-01-01
5,1970-01-01,1970-01-01 00:00:00.000000000,1970-01-01


In [14]:
C.sort_values(["E", "Dates", "Dates_Right"], ignore_index=True)

NameError: name 'C' is not defined

In [ ]:
B.sort_values(["E", "Dates", "Dates_Right"], ignore_index=True)

In [ ]:
C = (
    df[["E"]]
    .merge(right[["Dates", "Dates_Right"]], how="cross")
    .loc[lambda df: df.E.between(df.Dates, df.Dates_Right, inclusive="both")]
)

C

In [ ]:
C.sort_values(["E", "Dates", "Dates_Right"], ignore_index=True)

In [ ]:
df.dtypes, right.dtypes

In [ ]:
(
    df[["A", "E"]].conditional_join(
        right[["Integers", "Dates"]],
        ("E", "Dates", "<"),
        ("A", "Integers", ">="),
        how="inner",
        use_numba=True,
        sort_by_appearance=False,
    )
)

In [ ]:
columns = ["A", "E", "Integers", "Dates"]
(
    df.merge(
        right,
        how="cross",
    )
    .loc[lambda df: df.A.ge(df.Integers) & df.E.lt(df.Dates), columns]
    .sort_values(columns, ignore_index=True)
)

In [ ]:
(
    df[["A", "E"]].conditional_join(
        right[["Integers", "Dates"]],
        ("E", "Dates", "<"),
        ("A", "Integers", ">="),
        how="inner",
        use_numba=False,
        sort_by_appearance=False,
    )
)

In [ ]:
print 'j'

In [ ]:
pd.__version__

In [ ]:
df1 = pd.DataFrame({"id": [1, 1, 1, 2, 2, 3], "value_1": [2, 5, 7, 1, 3, 4]})
df2 = pd.DataFrame(
    {
        "id": [1, 1, 1, 1, 2, 2, 2, 3],
        "value_2A": [0, 3, 7, 12, 0, 2, 3, 1],
        "value_2B": [1, 5, 9, 15, 1, 4, 6, 3],
    }
)

In [ ]:
df1 = pd.concat([df1] * 20000)
df2 = pd.concat([df2] * 300)

df1.conditional_join(
    df2,
    ("value_1", "value_2A", ">"),
    ("value_1", "value_2B", "<"),
    use_numba=True,
)

In [ ]:
df1.conditional_join(
    df2,
    ("value_1", "value_2A", ">"),
    ("value_1", "value_2B", "<"),
    use_numba=False,
)

In [ ]:
%timeit df1.conditional_join(df2, ('value_1','value_2A','>'), ('value_1','value_2B','<'),use_numba=True)

In [ ]:
# 32.6ms   30_000_000

In [ ]:
df1

In [ ]:
# events = pd.read_csv(
#     "/home/sam/Downloads/results.csv", parse_dates=["start", "end"]
# ).iloc[:, 1:]
# events.head()

In [ ]:
events = pd.read_csv(
    "/Users/samuel.oranyeli/Downloads/results.csv",
    parse_dates=["start", "end"],
).iloc[:, 1:]

In [ ]:
a = 1
a.max()

AttributeError: 'int' object has no attribute 'max'

In [ ]:
A = events.conditional_join(
    events,
    ("start", "end", "<="),
    ("end", "start", ">="),
    # ('id', 'id', '!='),
    # ('audience','audience','>'),
    use_numba=True,
    df_columns=["id", "start", "end"],
    right_columns=["id", "start", "end"],
)

A

pristine start
[[    0 28282]
 [    1 28281]
 [    2 28280]
 ...
 [28280     2]
 [28281     1]
 [28282     0]]
[[    0 28282]
 [    1 28281]
 [    2 28280]
 ...
 [28280     2]
 [28281     1]
 [28282     0]]
pristine end


(array([    0,     1,     2, ..., 29996, 29997, 29998]), 29999)

In [ ]:
B = events.conditional_join(
    events,
    ("start", "end", "<="),
    ("end", "start", ">="),
    # ('id', 'id', '!='),
    # ('audience','audience','>'),
    use_numba=False,
    df_columns=["id", "start", "end"],
    right_columns=["id", "start", "end"],
)

In [ ]:
A.sort_values(A.columns.tolist(), ignore_index=True).equals(
    B.sort_values(A.columns.tolist(), ignore_index=True)
)

In [ ]:
%%timeit
(
    events.conditional_join(
        events,
        ("start", "end", "<="),
        ("end", "start", ">="),
        # ('id', 'id', '!='),
        # ('audience','audience','>'),
        use_numba=True,
        df_columns=["id", "start", "end"],
        right_columns=["id", "start", "end"],
    )
)

In [ ]:
%%timeit
(
    events.conditional_join(
        events,
        ("start", "end", "<="),
        ("end", "start", ">="),
        # ('id', 'id', '!='),
        # ('audience','audience','>'),
        use_numba=False,
        df_columns=["id", "start", "end"],
        right_columns=["id", "start", "end"],
    )
)

In [ ]:
C = dict(
    key=[f"c{num}" for num in range(1, 8)],
    vol=[35, 15, 5, 35, 18, 90, 17],
    profit=[45, 35, 55, 12, 15, 55, 11],
    unitsSold=[15, 10, 30, 10, 15, 80, 2],
    keyy=range(1, 8),
)
C = pd.DataFrame(C)


D = dict(
    key=[f"d{num}" for num in range(1, 9)],
    vol=[20, 50, 15, 16, 40, 20, 40, 2],
    profit=[30, 10, 12, 52, 35, 20, 30, 57],
    unitsSold=[20, 35, 10, 12, 40, 30, 5, 15],
    keyy=range(8, 0, -1),
)
D = pd.DataFrame(D)

In [ ]:
C = pd.concat([C] * 400)
D = pd.concat([D] * 400)

In [ ]:
C.conditional_join(
    D,
    ("profit", "profit", ">="),
     ('keyy','keyy','<='),
      ('unitsSold','unitsSold','>='),
    # ("vol", "vol", "<="),
      ('vol','profit','>'),
    use_numba=True,
)

left                           right                          
   key vol profit unitsSold keyy   key vol profit unitsSold keyy
0   c6  90     55        80    6    d1  20     30        20    8
1   c6  90     55        80    6    d3  15     12        10    6
2   c6  90     55        80    6    d2  50     10        35    7
3   c1  35     45        15    1    d7  40     30         5    2
4   c1  35     45        15    1    d3  15     12        10    6
5   c2  15     35        10    2    d3  15     12        10    6
6   c4  35     12        10    4    d3  15     12        10    6
7   c5  18     15        15    5    d3  15     12        10    6

In [ ]:
C.conditional_join(
    D,
    ("profit", "profit", ">="),
     ('keyy','keyy','<='),
      ('unitsSold','unitsSold','>='),
    # ("vol", "vol", "<="),
      ('vol','profit','>'),
    use_numba=False,
)

left                           right                          
   key vol profit unitsSold keyy   key vol profit unitsSold keyy
0   c1  35     45        15    1    d3  15     12        10    6
1   c1  35     45        15    1    d7  40     30         5    2
2   c2  15     35        10    2    d3  15     12        10    6
3   c4  35     12        10    4    d3  15     12        10    6
4   c5  18     15        15    5    d3  15     12        10    6
5   c6  90     55        80    6    d2  50     10        35    7
6   c6  90     55        80    6    d3  15     12        10    6
7   c6  90     55        80    6    d1  20     30        20    8

In [ ]:
%%timeit
out = C.conditional_join(
    D,
    ("profit", "profit", ">="),
     ('keyy','keyy','<='),
      ('unitsSold','unitsSold','>='),
    # ("vol", "vol", "<="),
      ('vol','profit','>'),
    use_numba=True,
)

8.59 ms ± 50.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%%timeit
C.conditional_join(
    D,
    ("profit", "profit", ">="),
     ('keyy','keyy','<='),
      ('unitsSold','unitsSold','>='),
    # ("vol", "vol", "<="),
      ('vol','profit','>'),
    use_numba=True,
)


758 ms ± 5.47 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%timeit
out = C.conditional_join(
    D,
    ("profit", "profit", ">="),
     ('keyy','keyy','<='),
      ('unitsSold','unitsSold','>='),
    # ("vol", "vol", "<="),
      ('vol','profit','>'),
    use_numba=False,
)



67.6 ms ± 546 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
C.conditional_join(
    D,
    ("profit", "profit", ">="),
     ('keyy','keyy','<='),
      ('unitsSold','unitsSold','>='),
    # ("vol", "vol", "<="),
      ('vol','profit','>'),
    use_numba=False,
)[0].size


In [ ]:
C.conditional_join(
    D,
    ("profit", "profit", ">="),
     ('keyy','keyy','<='),
      ('unitsSold','unitsSold','>='),
    # ("vol", "vol", "<="),
      ('vol','profit','>'),
    use_numba=False,
)

In [ ]:
(
    C.conditional_join(
        D,
        ("profit", "profit", ">="),
        # ('keyy','keyy','=='),
          # ('unitsSold','unitsSold','>='),
        ("vol", "vol", "<="),
        # ('vol','profit','>'),
        use_numba=False,
    )
)

In [ ]:
%%timeit
(
    C.conditional_join(
        D,
        ("profit", "profit", ">="),
        # ('keyy','keyy','=='),
        #  ('unitsSold','unitsSold','>='),
        ("vol", "vol", "<="),
        # ('vol','profit','>'),
        use_numba=True,
    )
)

In [ ]:
%%timeit
(
    C.conditional_join(
        D,
        ("profit", "profit", ">="),
        # ('keyy','keyy','=='),
        #  ('unitsSold','unitsSold','>='),
        ("vol", "vol", "<="),
        # ('vol','profit','>'),
        use_numba=False,
    )
)

In [ ]:
print 'i'

In [ ]:
C = pd.concat([C] * 400)
D = pd.concat([D] * 400)

In [ ]:
(
    C.conditional_join(
        D,
        ("profit", "profit", ">="),
        # ('keyy','keyy','=='),
        # ('unitsSold','unitsSold','>='),
        ("vol", "vol", "<="),
        # ('vol','profit','>'),
        use_numba=True,
    )
)

In [ ]:
%%timeit
(
    C.conditional_join(
        D,
        ("profit", "profit", ">="),
        # ('keyy','keyy','=='),
        # ('unitsSold','unitsSold','>='),
        ("vol", "vol", "<="),
        # ('vol','profit','>'),
        use_numba=True,
    )
)

In [ ]:
%%timeit
(
    C.conditional_join(
        D,
        ("profit", "profit", ">="),
        # ('keyy','keyy','=='),
        # ('unitsSold','unitsSold','>='),
        ("vol", "vol", "<="),
        # ('vol','profit','>'),
        use_numba=False,
    )
)

In [ ]:
(
    C.conditional_join(
        D,
        ("profit", "profit", ">="),
        # ('keyy','keyy','=='),
        # ('unitsSold','unitsSold','>='),
        ("vol", "vol", "<="),
        # ('vol','profit','>'),
        use_numba=True,
    )
)[0].size

In [ ]:
np.asarray(1), np.asanyarray(1)

In [ ]:
(
    C.conditional_join(
        D,
        ("profit", "profit", ">="),
        # ('keyy','keyy','=='),
        # ('unitsSold','unitsSold','>='),
        ("vol", "vol", "<="),
        # ('vol','profit','>'),
        use_numba=False,
    )
)[0].size

In [ ]:
from string import ascii_lowercase

np.random.seed(1)
# n = 10000000; k = 20000
n = 20_0_000
k = 20_00
mapp = {k: v for v, k in enumerate(ascii_lowercase)}
idx1 = np.random.randint(0, high=2_00, size=n)
idx2 = np.random.randint(0, high=3_00, size=n)

d1 = dict(
    x=np.random.choice(list(ascii_lowercase[:5]), size=n),
    start=np.minimum(idx1, idx2),
    end=np.maximum(idx1, idx2),
)


d2 = dict(
    x=np.random.choice(list(ascii_lowercase[:15]), size=k),
    pos1=np.random.randint(low=60, high=151, size=k),
)

d1 = pd.DataFrame(d1)
d2 = pd.DataFrame(d2)
d1 = d1.assign(xx=lambda df: df.x.map(mapp))
d2 = d2.assign(xx=lambda df: df.x.map(mapp))

In [ ]:
%%timeit
bla = d2.conditional_join(
    d1,
    ("pos1", "start", ">"),
    ("pos1", "end", "<"),
    # ('xx', 'xx', '=='),
    use_numba=True,
)

In [ ]:
%%timeit
bla = d2.conditional_join(
    d1,
    ("pos1", "start", ">"),
    ("pos1", "end", "<"),
    # ('xx', 'xx', '=='),
    use_numba=False,
)

In [ ]:
%%timeit
bla = d2.conditional_join(
    d1,
    ("pos1", "start", ">"),
    ("pos1", "end", "<"),
    # ('xx', 'xx', '=='),
    use_numba=False,
)

In [ ]:
print 'jarr'

In [ ]:
employees = pd.read_csv("/Users/samuel.oranyeli/employees.csv")

In [ ]:
%%timeit
(
    employees.conditional_join(
        employees, ("salary", "salary", "<"), ("tax", "tax", ">"), use_numba=True
    )
)

In [ ]:
%%timeit
(
    employees.conditional_join(
        employees, ("salary", "salary", "<"), ("tax", "tax", ">"), use_numba=False
    )
)